In [ ]:
#--------------------
# Necessary Libraries
#--------------------
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler

In [ ]:
# pip install wandb

In [ ]:
#----------------------
# Log in to W&B account
#----------------------
import wandb
wandb.login(key='150002a34bcf7d04848ccaff65ab76ca5cc3f11b')

In [ ]:
#-------------------------
# Inaturalist-dataset path
#-------------------------
data_dir = "/kaggle/input/inaturalist-dataset/nature_12K/inaturalist_12K"

In [ ]:
#---------------------------
# Show Images from each Class
#---------------------------
import os
import matplotlib.pyplot as plt

dataset_path = "/kaggle/input/inaturalist-dataset/nature_12K/inaturalist_12K/test"

for class_name in sorted(os.listdir(dataset_path)):
    class_dir = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_dir):
        img_path = os.path.join(class_dir, os.listdir(class_dir)[2])
        img = plt.imread(img_path)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Part A: Training from scratch

## Question 1
Build a small CNN model consisting of $5$ convolution layers. Each convolution layer would be followed by an activation and a max-pooling layer. 

After $5$ such conv-activation-maxpool blocks, you should have one dense layer followed by the output layer containing $10$ neurons ($1$ for each of the $10$ classes). The input layer should be compatible with the images in the [iNaturalist dataset](https://storage.googleapis.com/wandb_datasets/nature_12K.zip) dataset.

In [ ]:
#---------------------
# Define the CNN model
#---------------------
class SimpleCNN(nn.Module):
    def __init__(self, input_channels, num_filters_list, filter_size, activation_func, dense_neurons, num_classes=10, use_batchnorm=False, dropout_rate=0.0):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.ModuleList()

        for i in range(5):   # Convolution Layers = 5
            self.conv_layers.append(
                nn.Conv2d(input_channels, num_filters_list[i], filter_size, stride=1, padding=0)
            )
            if use_batchnorm:
                self.conv_layers.append(nn.BatchNorm2d(num_filters_list[i]))
            self.conv_layers.append(activation_func())
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=1, padding=0))
            input_channels = num_filters_list[i]

        # Calculate output size of convolutional layers
        output_size = self.calculate_output_size(224)  # Assuming input size is 224x224

        self.dense_layer = nn.Linear(input_channels * output_size * output_size, dense_neurons)
        self.output_layer = nn.Linear(dense_neurons, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        
    def calculate_output_size(self, input_size):
        output_size = input_size
        for layer in self.conv_layers:
            if isinstance(layer, nn.Conv2d):
                output_size = (output_size - layer.kernel_size[0] + 2 * layer.padding[0]) // layer.stride[0] + 1
            elif isinstance(layer, nn.MaxPool2d):
                output_size = (output_size - layer.kernel_size + 2 * layer.padding) // layer.stride + 1
        return output_size

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)  # dropout before the dense layer
        x = self.dense_layer(x)
        x = self.output_layer(x)
        return x

In [ ]:
#-------------------------------
# Data Loading and Preprocessing
#-------------------------------
def load_and_preprocess_data(data_dir, batch_size, validation_split=0.2):
    """
    Loads and preprocesses the iNaturalist dataset.

    Args:
        data_dir (str): The directory containing the train and test folders.
        batch_size (int): The batch size for training and validation.
        validation_split (float): The proportion of the training data to use for validation.

    Returns:
        tuple: DataLoaders for the training and validation sets.
    """

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Standardize the pixel values
    ])

    train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
    test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)

    # Create data indices for the training and validation splits
    dataset_size = len(train_dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    np.random.shuffle(indices) # Shuffle the indices
    train_indices, val_indices = indices[split:], indices[:split]

    # Create samplers for the training and validation datasets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, test_dataset

In [ ]:
#-----------------------
# Training the CNN model
#-----------------------
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
    """
    Trains the CNN model.

    Args:
        model (nn.Module): The CNN model.
        train_loader (DataLoader): DataLoader for the training set.
        val_loader (DataLoader): DataLoader for the validation set.
        optimizer (optim.Optimizer): The optimizer.
        criterion (nn.Module): The loss function.
        epochs (int): The number of epochs to train for.
        device (torch.device): The device to use for training (CPU or GPU).
    """
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Train loss and Accuracy
            train_loss += loss.item() * labels.size(0)  # Accumulate loss
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
        avg_train_loss = train_loss / total
        train_accuracy = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
        avg_val_loss = val_loss / val_total
        val_accuracy = val_correct / val_total

        # Log in wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": avg_val_loss,
            "val_accuracy": val_accuracy
        })
        print(f"Epoch [{epoch+1}/{epochs}], Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
#---------------------------------------------------------------------
# Function to convert string activation to PyTorch activation function
#---------------------------------------------------------------------
def get_activation(activation_str):
    if activation_str == 'ReLU':
        return nn.ReLU
    elif activation_str == 'GELU':
        return nn.GELU
    elif activation_str == 'SiLU':
        return nn.SiLU
    elif activation_str == 'Mish':
        return nn.Mish
    else:
        raise ValueError(f"Invalid activation function: {activation_str}")


In [ ]:
#--------------------------
# Wandb Sweep Configuration
#---------------------------
sweep_config = {
    "method": "bayes",  # Bayesian optimization for efficiency
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'   
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3  # At least 3 epochs before being evaluated for early stopping
    },
    'parameters': {
        'epochs': {
            "values": [5, 10]
        },
        'num_filters': {
            'values': [32, 64]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'filter_organization': {
            'values': ['same', 'doubling', 'halving']
        },
        'data_augmentation': {
            'values': [True, False]
        },
        'batch_norm': {
            'values': [True, False]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        }
    }
}

In [ ]:
#---------------------
# Wandb Sweep Function
#---------------------
def wandb_sweep():
    """
    Performs a hyperparameter sweep using Wandb.
    """

    wandb.init(project="iNaturalist-CNN")

    # Access sweep configuration from wandb
    config = wandb.config

    # Run name
    run_name = f"ep-{config.epochs}_hf-{config.num_filters}_ac-{config.activation}_fo-{config.filter_organization}_da-{config.data_augmentation}_bn-{config.batch_norm}_dro-{config.dropout}_bs-{config.batch_size}_lr-{config.learning_rate}"
    wandb.run.name = run_name

    # Data Loading
    data_dir = "/kaggle/input/inaturalist-dataset/nature_12K/inaturalist_12K"
    train_loader, val_loader, test_dataset = load_and_preprocess_data(data_dir, config.batch_size)

    # Determine filter list based on organization
    if config.filter_organization == 'same':
        num_filters_list = [config.num_filters] * 5
    elif config.filter_organization == 'doubling':
        num_filters_list = [config.num_filters * (2**i) for i in range(5)]
    elif config.filter_organization == 'halving':
        num_filters_list = [config.num_filters // (2**i) if config.num_filters // (2**i) > 0 else 1 for i in range(5)]
    else:
        raise ValueError("Invalid filter organization")

    # Model Initialization
    activation_func = get_activation(config.activation)
    model = SimpleCNN(3, num_filters_list, 3, activation_func, 128, 10, config.batch_norm, config.dropout)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    # Training
    train(model, train_loader, val_loader, optimizer, criterion, epochs=config.epochs, device=device)

    # Evaluate on test set
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="iNaturalist-CNN")
    wandb.agent(sweep_id, wandb_sweep, count=29)

### **Best Hyperparameter Configuration**

```yaml
activation:
    value: SiLU
batch_norm:
    value: True
batch_size:
    value: 64
data_augmentation:
    value: False
dropout:
    value: 0.3
epochs:
    value: 5
filter_organization:
    value: doubling
learning_rate:
    value: 0.0001
num_filters:
    value: 64
```